ÍNDICE:

1. Protocolo de transporte de hipertexto-HTTP
2. El navegador web más sencillo del mundo
3. Recepción de una imagen mediante HTTP
4. Recepción de páginas web con urllib
5. Leyendo archivos binarios con urllib
6. Análisis de HTML y rascado de la web
7. Análisis de HTML mediante expresiones regulares
8. Análisis de HTML mediante BeautifulSoup
9. Sección extra para usuarios de Unix/Linux
10. Glosario
11. Ejercicios

In [2]:
import socket
import time
import urllib.request, urllib.parse, urllib.error
from urllib.request import urlopen
import re
import ssl
from bs4 import BeautifulSoup
import os

# 1. Protocolo de transporte de hipertexto-HTTP

# 2. El navegador web más sencillo del mundo

In [2]:
misock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
misock.connect(('data.pr4e.org',80))

cmd = 'GET http://data.pr4e.org/romeo.txt HTTP/1.0\r\n\r\n'.encode()
misock.send(cmd)

while True:
    datos = misock.recv(512)
    if len(datos) < 1:
        break
    print(datos.decode(),end='')

misock.close()

HTTP/1.1 200 OK
Date: Sun, 10 Dec 2023 00:42:31 GMT
Server: Apache/2.4.18 (Ubuntu)
Last-Modified: Sat, 13 May 2017 11:22:22 GMT
ETag: "a7-54f6609245537"
Accept-Ranges: bytes
Content-Length: 167
Cache-Control: max-age=0, no-cache, no-store, must-revalidate
Pragma: no-cache
Expires: Wed, 11 Jan 1984 05:00:00 GMT
Connection: close
Content-Type: text/plain

But soft what light through yonder window breaks
It is the east and Juliet is the sun
Arise fair sun and kill the envious moon
Who is already sick and pale with grief


In [3]:
b'Hola mundo'

b'Hola mundo'

In [4]:
'Hola mundo'.encode()

b'Hola mundo'

# 3. Recepción de una imagen mediante HTTP

In [6]:
SERVIDOR = 'data.pr4e.org'
PUERTO = 80

misock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
misock.connect((SERVIDOR, PUERTO))
misock.sendall(b'GET http://data.pr4e.org/cover3.jpg HTTP/1.0\r\n\r\n')

contador = 0
imagen = b""

while True:
    datos = misock.recv(5120)
    if len(datos) < 1:
        break
    time.sleep(0.25)
    contador += len(datos)
    print(len(datos),contador)
    imagen += datos
    
misock.close()

# Búsqueda del final de la cabecera (2 CRLF)
pos = imagen.find(b"\r\n\r\n")
print('Longitud del encabezado',pos)
print(imagen[:pos].decode())

# Ignorar la cabecera y guardar los datos de la imagen
imagen = imagen[pos+4:]

manejador_archivos = open("cosa.jpg","wb")

manejador_archivos.write(imagen)

manejador_archivos.close()

5120 5120
5120 10240
5120 15360
5120 20480
5120 25600
5120 30720
5120 35840
5120 40960
5120 46080
5120 51200
5120 56320
5120 61440
5120 66560
5120 71680
5120 76800
5120 81920
5120 87040
5120 92160
5120 97280
5120 102400
5120 107520
5120 112640
5120 117760
5120 122880
5120 128000
5120 133120
5120 138240
5120 143360
5120 148480
5120 153600
5120 158720
5120 163840
5120 168960
5120 174080
5120 179200
5120 184320
5120 189440
5120 194560
5120 199680
5120 204800
5120 209920
5120 215040
5120 220160
5120 225280
5120 230400
208 230608
Longitud del encabezado 394
HTTP/1.1 200 OK
Date: Sun, 10 Dec 2023 00:46:06 GMT
Server: Apache/2.4.18 (Ubuntu)
Last-Modified: Mon, 15 May 2017 12:27:40 GMT
ETag: "38342-54f8f2e5b6277"
Accept-Ranges: bytes
Content-Length: 230210
Vary: Accept-Encoding
Cache-Control: max-age=0, no-cache, no-store, must-revalidate
Pragma: no-cache
Expires: Wed, 11 Jan 1984 05:00:00 GMT
Connection: close
Content-Type: image/jpeg


# 4. Recepción de páginas web con urllib

In [8]:
manejador_archivos = urllib.request.urlopen('http://data.pr4e.org/romeo.txt')
for linea in manejador_archivos:
    print(linea.decode().strip())

But soft what light through yonder window breaks
It is the east and Juliet is the sun
Arise fair sun and kill the envious moon
Who is already sick and pale with grief


In [10]:
manejador_archivos = urllib.request.urlopen('http://data.pr4e.org/romeo.txt')

contadores = dict()

for linea in manejador_archivos:
    palabras = linea.decode().split()
    for palabra in palabras:
        contadores[palabra] = contadores.get(palabra, 0) + 1

print(contadores)

{'But': 1, 'soft': 1, 'what': 1, 'light': 1, 'through': 1, 'yonder': 1, 'window': 1, 'breaks': 1, 'It': 1, 'is': 3, 'the': 3, 'east': 1, 'and': 3, 'Juliet': 1, 'sun': 2, 'Arise': 1, 'fair': 1, 'kill': 1, 'envious': 1, 'moon': 1, 'Who': 1, 'already': 1, 'sick': 1, 'pale': 1, 'with': 1, 'grief': 1}


# 5. Leyendo archivos binarios con urllib

In [11]:
img = urllib.request.urlopen('http://data.pr4e.org/cover3.jpg').read()

manejador_archivos = open('portada.jpg','wb')
manejador_archivos.write(img)
manejador_archivos.close()

In [12]:
img = urllib.request.urlopen('http://data.pr4e.org/cover3.jpg')

manejador_archivos = open('portada.jpg','wb')
tamano = 0
while True:
    info = img.read(100000)
    if len(info) < 1:
        break
    tamano += len(info)
    manejador_archivos.write(info)
    
print(tamano, 'caracteres copiados.')
manejador_archivos.close()

230210 caracteres copiados.


# 6. Análisis de HTML y rascado de la web

# 7. Análisis de HTML mediante expresiones regulares

In [14]:
# Ignorar errores de certificado SSL

ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

url = input('Introduzca - ')
html = urllib.request.urlopen(url).read()
enlaces = re.findall(b'href="(http[s]?://.*?)"', html)

for enlace in enlaces:
    print(enlace.decode())

Introduzca - https://docs.python.org
https://docs.python.org/3/index.html
https://www.python.org/
https://docs.python.org/3.13/
https://docs.python.org/3.12/
https://docs.python.org/3.11/
https://docs.python.org/3.10/
https://docs.python.org/3.9/
https://docs.python.org/3.8/
https://docs.python.org/3.7/
https://docs.python.org/3.6/
https://docs.python.org/3.5/
https://docs.python.org/3.4/
https://docs.python.org/3.3/
https://docs.python.org/3.2/
https://docs.python.org/3.1/
https://docs.python.org/3.0/
https://docs.python.org/2.7/
https://docs.python.org/2.6/
https://www.python.org/doc/versions/
https://peps.python.org
https://wiki.python.org/moin/BeginnersGuide
https://wiki.python.org/moin/PythonBooks
https://www.python.org/doc/av/
https://devguide.python.org/
https://www.python.org/
https://devguide.python.org/docquality/#helping-with-documentation
https://docs.python.org/3.13/
https://docs.python.org/3.12/
https://docs.python.org/3.11/
https://docs.python.org/3.10/
https://docs.pyth

# 8. Análisis de HTML mediante BeautifulSoup

In [16]:
# Ignorar errores de certificado SSL
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

url = input('Introduzca - ')
html = urllib.request.urlopen(url, context=ctx).read()
sopa = BeautifulSoup(html, 'html.parser')

# Recuperar todas las etiquetas de anclaje
etiquetas = sopa('a')
for etiqueta in etiquetas:
    print(etiqueta.get('href', None))

Introduzca - https://docs.python.org
https://www.python.org/
download.html
https://docs.python.org/3.13/
https://docs.python.org/3.12/
https://docs.python.org/3.11/
https://docs.python.org/3.10/
https://docs.python.org/3.9/
https://docs.python.org/3.8/
https://docs.python.org/3.7/
https://docs.python.org/3.6/
https://docs.python.org/3.5/
https://docs.python.org/3.4/
https://docs.python.org/3.3/
https://docs.python.org/3.2/
https://docs.python.org/3.1/
https://docs.python.org/3.0/
https://docs.python.org/2.7/
https://docs.python.org/2.6/
https://www.python.org/doc/versions/
https://peps.python.org
https://wiki.python.org/moin/BeginnersGuide
https://wiki.python.org/moin/PythonBooks
https://www.python.org/doc/av/
https://devguide.python.org/
genindex.html
py-modindex.html
https://www.python.org/
#

whatsnew/3.12.html
whatsnew/index.html
tutorial/index.html
library/index.html
reference/index.html
using/index.html
howto/index.html
installing/index.html
distributing/index.html
extending/inde

In [18]:
# Ignorar errores de certificado SSL
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

url = input('Introduzca - ')
html = urlopen(url, context=ctx).read()
sopa = BeautifulSoup(html, "html.parser")

# Obtener todas las etiquetas de anclaje
etiquetas = sopa('a')
for etiqueta in etiquetas:
    print('ETIQUETA:', etiqueta)
    print('URL:', etiqueta.get('href',None))
    print('Contenido:', etiqueta.contents[0])
    print('Atributos:', etiqueta.attrs)

Introduzca - http://www.dr-chuck.com/page1.html
ETIQUETA: <a href="http://www.dr-chuck.com/page2.htm">
Second Page</a>
URL: http://www.dr-chuck.com/page2.htm
Contenido: 
Second Page
Atributos: {'href': 'http://www.dr-chuck.com/page2.htm'}


# 9. Sección extra para usuarios de Unix/Linux

In [21]:
print('Ingresa una URL como http://data.pr4e.org/cover3.jpg')

urlstr = input().strip()
img = urllib.request.urlopen(urlstr)

# Obtener la última "palabra"
palabras = urlstr.split('/')
nombre_archivo = palabras[-1]

# No sobreescribir el archivo
if os.path.exists(nombre_archivo):
    if input('¿Remplazar ' + nombre_archivo + ' (S/n)?') != 'S':
        print('Datos no copiados')
        exit()
    print('Reemplazando', nombre_archivo)

manejador_archivos = open(nombre_archivo, 'wb')
longitud = 0

while True:
    info = img.read(100000)
    if len(info) < 1: break
    longitud = longitud + len(info)
    manejador_archivos.write(info)

print(longitud, 'caracteres copiados en', nombre_archivo)
manejador_archivos.close()

Ingresa una URL como http://data.pr4e.org/cover3.jpg
http://www.dr-chuck.com/page1.html
¿Remplazar page1.html (S/n)?n
Datos no copiados
Reemplazando page1.html
131 caracteres copiados en page1.html


# 10. Glosario

BeautifulSoup: una librería de Python para analizar documentos HTML y extraer datos de ellos, que compensa la mayoría de las imperfecciones que los navegadores HTML normalmente ignoran. Puedes descargar el código de BeautifulSoup desde www.crummy.com.

Puerto: un número que generalmente indica qué aplicación estás contactando cuando realizas una conexión con un socket en un servidor. Por ejemplo, el tráfico web normalmente usa el puerto 80, mientras que el tráfico del correo electrónico usa el puerto 25.

Rascado: cuando un programa simula ser un navegador web y recupera una página web, para luego realizar una búsqueda en su contenido. A menudo los programas siguen los enlaces en una página para encontrar la siguiente, de modo que pueden atravesar una red de páginas o una red social.

Rastrear: la acción de un motor de búsqueda web que consiste en recuperar una página y luego todas las páginas enlazadas por ella, continuando así sucesivamente hasta que tienen casi todas las páginas de Internet, que usan a continuación para construir su índice de búsqueda.

Socket: una conexión de red entre dos aplicaciones, en la cual dichas aplicaciones pueden enviar y recibir datos en ambas direcciones.

# 11. Ejercicios

1. Utiliza urllib para rehacer el ejercicio anterior de modo que 

(1) reciba el documento de una URL, 

(2) muestre hasta 3000 caracteres, y 

(3) cuente la cantidad total de caracteres en el documento. 

No te preocupes de las cabeceras en este ejercicio, simplemente muesta los primeros 3000 caracteres del contenido del documento.

In [3]:
manejador_archivos = urllib.request.urlopen('http://data.pr4e.org/romeo.txt')

caracteres = 0
caracteres_limit = 3000

for linea in manejador_archivos:
    linea = linea.decode()
    next_count = caracteres + len(linea)
    if next_count <= caracteres_limit:
        print(linea.rstrip('\n'))
    elif caracteres < caracteres_limit:
        caracteres_remain = caracteres_limit - caracteres - 1
        print(linea[:caracteres_remain])
    caracteres = next_count
print(caracteres)

But soft what light through yonder window breaks
It is the east and Juliet is the sun
Arise fair sun and kill the envious moon
Who is already sick and pale with grief
167


2. Cambia el programa urllinks.py para extraer y contar las etiquetas de párrafo (p) del documento HTML recuperado y mostrar el total de párrafos como salida del programa. No muestres el texto de los párrafos, sólo cuéntalos. Prueba el programa en varias páginas web pequeñas, y también en otras más grandes.

In [4]:
contador = 0                               

ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

url = input('Ingrese - ')
html = urllib.request.urlopen(url, context=ctx).read()
sopa = BeautifulSoup(html, 'html.parser')

# Recuperar todas las etiquetas de anclaje
tags = sopa('p')

for tag in tags:
    contador += 1  

print(contador)

Ingrese - http://data.pr4e.org/romeo.txt
0


In [6]:
sopa

But soft what light through yonder window breaks
It is the east and Juliet is the sun
Arise fair sun and kill the envious moon
Who is already sick and pale with grief

3. Cambia el programa del socket de modo que solamente muestra datos después de que se haya recibido la cabecera y la línea en blanco. Recuerda que recv recibe caracteres (saltos de línea incluidos), no líneas.

In [7]:
my_sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
my_sock.connect(('data.pr4e.org', 80))
cmd = 'GET http://data.pr4e.org/romeo.txt HTTP/1.0\r\n\r\n'.encode()
my_sock.send(cmd)

datos = my_sock.recv(512)
mensaje = datos.decode()
header_end_pos = mensaje.find('\r\n\r\n') + 4   # Encuentra el final del encabezado

# Agrega cuatro para excluir: '\r\n\r\n'
print(mensaje[header_end_pos:], end='')

while True:                                 # Encabezado en los primeros datos solamente
    datos = my_sock.recv(512)
    if not datos:
        break
    print(datos.decode())

my_sock.close()

But soft what light through yonder window breaks
It is the east and Juliet is the sun
Arise fair sun and kill the envious moon
Who is already sick and pale with grief

